# 畳み込みニューラルネットワーク (CNN)

![](./capture/ml_lesson_7_1_capture01.png)

## Section2 CNNを使ったモデルを構築してみよう

In [ ]:
%matplotlib inline

import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

random_state = 42

### 2.1 Fashion MNIST(白黒画像)をCNNでクラス分類

#### 2.1.1 データセットの読み込み

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
fig = plt.figure(figsize=(9, 15))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05,
                    wspace=0.05)

for i in range(9):
    ax = fig.add_subplot(1, 9, i + 1, xticks=[], yticks=[])
    ax.imshow(x_train[i], cmap='gray')

このとき読み込んだ画像は(バッチサイズ、縦の画素数、 横の画素数)の次元で表されています。

In [ ]:
x_train.shape

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

前章の多層パーセプトロンでは入力を (バッチサイズ、画素数) の2次元テンソルとして扱いましたが、 CNNでは2次元の画像として処理していくために4次元テンソル (バッチサイズ、縦の画素数、横の画素数、チャンネル数)として扱います。 チャンネル数は白黒画像の場合は1、 カラー画像の場合はRGBで3です。

Fashion MNISTの画像は白黒データですのでチャンネル数を1に設定しています。(カラー画像の場合はチャンネル数が3になります)

In [ ]:
print(x_train.shape)
print(y_train.shape)

#### 2.1.2 実装

In [ ]:
model = Sequential()

# 入力画像 28x28x1 (縦の画素数)x(横の画素数)x(チャンネル数)
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(28, 28, 1)))  # 28x28x1 -> 24x24x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 24x24x16 -> 12x12x16
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 12x12x16 -> 8x8x64
model.add(MaxPooling2D(pool_size=(2, 2)))  # 8x8x64 -> 4x4x64

model.add(Flatten())  # 4x4x64-> 1024
model.add(Dense(10, activation='softmax'))  # 1024 -> 10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

作成したモデルを確認してみましょう。

In [ ]:
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))
model.summary()

In [ ]:
# 学習を実行
# 実行時間の都合上、 epoch 数＝3 とする
# EarlyStopping：　過学習を防止するための仕組み。学習の精度が上がらなくなった段階で実行を停止する
early_stopping = EarlyStopping(patience=1, verbose=1)
#model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=1,
#          validation_split=0.1, callbacks=[early_stopping])
model.fit(x=x_train, y=y_train, batch_size=32, epochs=3, verbose=1,
          validation_split=0.1, callbacks=[early_stopping])

# 性能評価（ loss：　予測結果と正解との誤差、accuracy：　正解率）
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### 2.2 CIFAR10のデータ(カラー画像)をCNNでクラス分類

####  2.2.1 データセットの読み込み

6万枚のカラー画像に10のカテゴリのどれかが付与されたCIFAR-10というデータセットを使用します。

まず、データを読み込みます。

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
y_train = np.eye(10)[y_train.astype('int32').flatten()]

x_test = x_test.astype('float32') / 255
y_test = np.eye(10)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=10000)

画像はRGBデータなのでFashion MNISTとは異なり、チャンネル数は3になります。

In [ ]:
print(x_train.shape)
print(y_train.shape)

次に、CIFAR-10の画像の例を表示してみます。この画像ひとつひとつに10のカテゴリのうちひとつが付与されています。

In [ ]:
fig = plt.figure(figsize=(9, 15))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05,
                    wspace=0.05)

for i in range(9):
    ax = fig.add_subplot(1, 9, i + 1, xticks=[], yticks=[])
    ax.imshow(x_train[i])

以下のネットワークを実装してみます。

![](./figures/lenet.png)

Y. LeCun et al., "Gradient-based learning applied to document recognition", Proceedings of the IEEE, 1998

#### 2.2.2 実装

In [ ]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  # 120 ->84
model.add(Dense(10, activation='softmax'))  # 84 ->10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

作成したモデルを確認してみましょう。

In [ ]:
#SVG(model_to_dot(model).create(prog='dot', format='svg'))
model.summary()

In [ ]:
# 学習を実行
# 実行時間の都合上、 epoch 数＝3 とする
# EarlyStopping：　過学習を防止するための仕組み。学習の精度が上がらなくなった段階で実行を停止する
early_stopping = EarlyStopping(patience=1, verbose=1)
#model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=1,
#          validation_data=(x_valid, y_valid), callbacks=[early_stopping])
model.fit(x=x_train, y=y_train, batch_size=32, epochs=3, verbose=1,
          validation_data=(x_valid, y_valid), callbacks=[early_stopping])

# 性能評価（ loss：　予測結果と正解との誤差、accuracy：　正解率）
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# ファインチューニング（ fine-tuning ）

## ファインチューニング、転移学習とは

![](./capture/ml_lesson_7_1_capture03.png)

## Keras で使える学習済みモデル

![](./capture/ml_lesson_7_1_capture02.png)

## CIFAR10のデータ(カラー画像)を MobileNet のファインチューニングでクラス分類

In [ ]:
# MobileNet モデルのライブラリを読み込む
from tensorflow.keras.applications.mobilenet import MobileNet

# 入力フォーマット
input_tensor = Input(shape=(32, 32, 3))
# MobileNet モデルを定義
base_model = MobileNet(include_top=False, weights='imagenet', input_tensor=input_tensor)

# 全結合層の新規構築
top_model = base_model.output
top_model = Flatten()(top_model)
#top_model = GlobalAveragePooling2D()(top_model) # Flatten の代わりに GlobalAveragePooling2D も有効であるとのこと
top_model = Dense(1024, activation = 'relu')(top_model)
predictions = Dense(10, activation = 'softmax')(top_model)

# ネットワーク全体の定義
model = Model(inputs = base_model.input, outputs = predictions)

# MobileNet の conv_dw_13 層の手前までは再学習させない
trainable = False
for layer in model.layers:
    if layer.name == 'conv_dw_13':
        trainable = True
    
    # conv_dw_13 以降の層は freeze 解除
    layer.trainable = trainable

    # ただし、conv_dw_13　以前の層でも、Batch Normalization 層は再学習させる
    if layer.name.endswith('bn'):
        layer.trainable = True

#
print("MobileNet {}層".format(len(model.layers)))

#
model.summary()

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# 学習を実行
# 実行時間の都合上、 epoch 数＝3 とする
# EarlyStopping：　過学習を防止するための仕組み。学習の精度が上がらなくなった段階で実行を停止する
early_stopping = EarlyStopping(patience=1, verbose=1)
#model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=1,
#          validation_data=(x_valid, y_valid), callbacks=[early_stopping])
model.fit(x=x_train, y=y_train, batch_size=32, epochs=3, verbose=1,
          validation_data=(x_valid, y_valid), callbacks=[early_stopping])

# 性能評価（ loss：　予測結果と正解との誤差、accuracy：　正解率）
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])